Run the config file to authenticate the machine and query variables from Key Vault

In [52]:
%run config

Code from file 'file:///c%3A/Users/nicholas.radich/Documents/Strava_Lakehouse/config.py':
 client_id = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientid") 
client_secret = dbutils.secrets.get(scope = "key_vault_secrets", key = "clientsecret") 
new_refresh_token = dbutils.secrets.get(scope = "key_vault_secrets", key = "newrefreshtoken")
activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "activityidpath") 
historical_activity_id_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "historicalactivitydfpath") 
segment_effort_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmenteffortpath") 
segment_details_path = dbutils.secrets.get(scope = "key_vault_secrets", key = "segmentdetailspath") 



import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"


payload = {
    'cli

Depedencies 

In [53]:
from pyspark.sql.functions import * 
from pyspark.sql.types import LongType
import pandas as pd
import requests 
import utils

API call to grab all of the acitivites within a personal account

In [ ]:
#API call to grab all of the acitivites within a personal account
def activity_api_call(access_token):
    """Returns all activities for a personal strava account, need access token"""
    activites_url = "https://www.strava.com/api/v3/athlete/activities"
    header = {'Authorization': 'Bearer ' + access_token}
    param = {'per_page': 200, 'page': 1}
    activity_dataset = requests.get(activites_url, headers=header, params=param).json()
    
    return activity_dataset


Make API call to get all strava activites

In [54]:
my_dataset = utils.activity_api_call(access_token)

Extracts activity ids and supporting information from initial dataset

In [ ]:
def extract_activities(dataset):
    """Function to seperate activity_ids and create an activity dataframe. 
    Returns a df of only the activity ids, and another df with more details about the activiity. """

    #Empty lists for columns we want to extract
    activity_ids = []
    start_date = []
    activity_name =[]
    distance = []
    moving_time = []
    elapsed_time = []
    sport_type = []
    total_elevation_gain =[]
    count = 0

    #a while loop to iterate through the dataset and append values to lists defined above
    while count < len(dataset):
        activity_ids.append(dataset[count]['id'])
        start_date.append(dataset[count]['start_date'])
        activity_name.append(dataset[count]['name'])
        distance.append(dataset[count]['distance'])
        moving_time.append(dataset[count]['moving_time'])
        elapsed_time.append(dataset[count]['elapsed_time'])
        sport_type.append(dataset[count]['sport_type'])
        total_elevation_gain.append(dataset[count]['total_elevation_gain'])
        count += 1 
        
    #convert list to dataframe   
    #create a DF of the soley the activity_ids
    activity_id_DF = spark.createDataFrame(activity_ids, LongType())
    #add file name and timestamp
    activity_id_DF = activity_id_DF.withColumnRenamed('value', 'activity_id')\
                                    .withColumn("ingest_file_name", lit("activity_ids")) \
                                    .withColumn("ingested_at", lit(current_timestamp()))
    
    #columns names for initial DF
    #need to specify schema
    columns = ['activity_ids','start_date', 'activity_name', 'distance', 'moving_time','elapsed_time', 'sport_type'\
          ,'total_elevation_gain']
    #list of lists
    #ccombine lists of extract values into list of list
    extracted_data = [activity_ids,start_date, activity_name, distance, moving_time,elapsed_time, sport_type\
          ,total_elevation_gain]

    #create a pandas Dataframe, then convert to spark to write to storage
    #create dataframe from list of list within specified column names
    pdf = pd.DataFrame.from_dict(dict(zip(columns, extracted_data)))
    activity_df = spark.createDataFrame(pdf)

    activity_df = activity_df.withColumn("ingest_file_name", lit("activity_information")) \
                             .withColumn("ingested_at", lit(current_timestamp()))

    return activity_id_DF, activity_df

Extract activity IDs and return refined columns 

In [55]:

activity_id_DF, activity_df = utils.extract_activities(my_dataset)

Function to write datasets to storage, delta format

In [ ]:
def write_dataframe_to_storage(dataset, storage_path, option, mode ):
    """Function to write activity ids to storage. Will overwrite current delta file in storage
    Option refers to schema overwriteSchema or mergeSchema, mode being either overwrite or append"""
    dataset.write.format("delta")\
    .option(option, "true")\
    .mode(mode)\
    .save(storage_path)

Write Activity_ID_DF to storage

In [56]:
#downside dataframes for testing purpose
activity_df= activity_df.limit(10)
activity_id_DF = activity_id_DF.limit(10)

In [57]:
#write the activity IDs to storage
utils.write_dataframe_to_storage(activity_id_DF,activity_id_path, "overwriteSchema","overwrite" )

Read the activities from storage

In [58]:
#read the activities from storage
stored_activity_ids = spark.read.format("delta").load(activity_id_path)

Compare what is currently in storage vs the most recent API call
If nothing is written to storage ie first run, still need to execute and write original DF to storage

In [ ]:

def get_historical_dataset(storagepath, historical_df_to_write, historical_storagepath):
    """Retrieve record from file path, if nothing exists, insert df to write to storage"""
    from pyspark.sql.utils import AnalysisException
    try:
        #try to read from storage
        historical_dataframe = spark.read.format("delta").load(historical_storagepath)
    except: 
        try:
        #if that fails, write the submitted dataframe to storage
            write_dataframe_to_storage(historical_df_to_write,storagepath, "mergeSchema", "append" )
        finally:
            historical_dataframe = spark.read.format("delta").load(storagepath)

    return historical_dataframe

Write full dataset to storage, will check to see if data already exists at path 

In [59]:
historical_activites = utils.get_historical_dataset(historical_activity_id_path, activity_df, historical_activity_id_path)

Extract those activites not currently in storage

In [60]:
#Extract activity ids from dataframes to list, to make comparison 
#activity_id_list = activity_id_DF.select('activity_ids').distinct().collect()
#from full dataset, all unique activity ids
activity_id_list = stored_activity_ids.select('activity_id').distinct().rdd.flatMap(lambda x: x).collect()

#currently what is written in storage, activity IDs
historical_activity_id_list = historical_activites.select('activity_ids').distinct().rdd.flatMap(lambda x: x).collect()

#make comparison between two, and find those IDs not written to storage
activity_ids_not_written_to_storage = [x for x in activity_id_list if x not in historical_activity_id_list ]

#filter original DF with those ids not currently in storage
new_activities = activity_df.filter(activity_df.activity_ids.isin(activity_ids_not_written_to_storage))

In [61]:
new_activities.show()

+------------+--------------------+------------------+--------+-----------+------------+----------+--------------------+--------------------+--------------------+
|activity_ids|          start_date|     activity_name|distance|moving_time|elapsed_time|sport_type|total_elevation_gain|    ingest_file_name|         ingested_at|
+------------+--------------------+------------------+--------+-----------+------------+----------+--------------------+--------------------+--------------------+
|  9709076171|2023-08-24T00:34:56Z|              laps|  2591.0|        901|         933|       Run|                 7.7|activity_information|2023-08-24 17:10:...|
|  9663381569|2023-08-16T23:33:23Z|Torrey Pines North|  4876.1|       3264|       10967|      Golf|                33.8|activity_information|2023-08-24 17:10:...|
|  9656452945|2023-08-16T00:06:24Z|      evening ride| 20579.3|       3888|        5220|      Ride|               158.6|activity_information|2023-08-24 17:10:...|
+------------+--------

In [62]:
#append new activity IDs to the storage location
utils.write_dataframe_to_storage(new_activities, historical_activity_id_path,"mergeSchema", "append")

In [ ]:
df1 = spark.read.format("delta").load(historical_activity_id_path)

In [ ]:
#only adds in distinct activities
from pyspark.sql import functions as F
df1.orderBy(F.desc('activity_ids')).show()

In [ ]:
activity_id_DF.select('activity_id').show()